In [1]:
%run '/mnt/berger2/equipes/IGE/meom/workdir/berger2/fonctions/meom_fonctions/fonctions.ipynb'

import numpy as np
import matplotlib.pyplot as plt
import scipy.io.netcdf as netcdf
import qgutils as qg
from matplotlib import ticker, cm
plt.ion()

In [2]:
#paramètres gravés dans le marbre pour le stage

Delta = 10000                               #grid spacing
f0 = 9.37456E-05                           #paramètre de Coriolis
dh = np.array([350,750,2900])              #hauteurs des couches océaniques
dh_b = np.array([(dh[0]+dh[1])/2,(dh[0]+dh[1])/2])
g_prime = np.array([2.5E-02,1.25E-02])     #gravités réduites aux interfaces
Beta = 1.75360E-11
delta_ek = 5                               #ocean bottom Ekman thickness 
nu4 = 2E09                                 #bi-harmonic viscosity
Ht = np.sum(dh)
Ht_b = np.sum(dh_b)
toc = np.array([1.31693E+01,1.81693E+01,2.41693E+01])
bf = delta_ek*f0/(2*dh[-1])

#fréquences Brunt-vaisala

N_12 = g_prime[0]/((dh[0]+dh[1])/2)
N_23 = g_prime[1]/((dh[1]+dh[2])/2)
N2 = np.array([N_12,N_23])

In [3]:
dir_abs  = '/mnt/berger2/equipes/IGE/meom/workdir/berger2/qgcm-data/double_gyre_coupled/outdata_7_dt10/'
dir_rel  = '/mnt/berger2/equipes/IGE/meom/workdir/berger2/qgcm-data/double_gyre_coupled/outdata_6tdiff_dt10/'

file0 = 'ocpo.nc'
file1 = 'ocsst.nc'

In [4]:
#get fields psi
pfile_abs = dir_abs + file0
pfile_rel = dir_rel + file0

toc_file_abs = dir_abs + file1
toc_file_rel = dir_rel + file1


f_abs = netcdf.netcdf_file(dir_abs + file0,'r')
f_rel = netcdf.netcdf_file(dir_rel + file0,'r')

In [5]:
N2,f0 = qg.reshape3d(dh,N2,f0)

si_t = f_abs.variables['time'][:].copy().size

p = qg.read_qgcm(pfile_abs, 0, var='p', rescale = 1/f0, interp = True, subtract_bc = False )
nz,ny,nx = p.shape

test = si_t

average = True

In [6]:
# flot moyen abs
flot_me = np.load('/mnt/berger2/equipes/IGE/meom/workdir/berger2/git/meom_test/meom_test/Lorenz_cycle/tableaux_termes_abs/termes_flot_moyen_abs.npz')
u_me_abs = flot_me['u_me']
v_me_abs = flot_me['v_me']

flot_me_bis = np.load('/mnt/berger2/equipes/IGE/meom/workdir/berger2/git/meom_test/meom_test/Lorenz_cycle/tableaux_termes_abs/etapes_flot_moyen_abs.npz')

b_me_abs = flot_me_bis['b_me']

#perturbations abs

flot_pert = np.load('/mnt/berger2/equipes/IGE/meom/workdir/berger2/git/meom_test//meom_test/Lorenz_cycle/cycle_termes_sep/tableaux_termes_exch/termes_abs.npz')

u_b_me_abs = flot_pert['u_b_me_abs']
v_b_me_abs = flot_pert['v_b_me_abs']

In [7]:
#flot moyen rel
flot_me = np.load('/mnt/berger2/equipes/IGE/meom/workdir/berger2/git/meom_test/meom_test/Lorenz_cycle/tableaux_termes_rel/termes_flot_moyen_rel.npz')
u_me_rel = flot_me['u_me']
v_me_rel = flot_me['v_me']

flot_me_bis = np.load('/mnt/berger2/equipes/IGE/meom/workdir/berger2/git/meom_test/meom_test/Lorenz_cycle/tableaux_termes_rel/etapes_flot_moyen_rel.npz')

b_me_rel = flot_me_bis['b_me']

flot_pert = np.load('/mnt/berger2/equipes/IGE/meom/workdir/berger2/git/meom_test//meom_test/Lorenz_cycle/cycle_termes_sep/tableaux_termes_exch/termes_rel.npz')

u_b_me_rel = flot_pert['u_b_me_rel']
v_b_me_rel = flot_pert['v_b_me_rel']

In [8]:
mpe2epe_abs = -b_me_abs*(div_x(u_b_me_abs, Delta)-div_y(v_b_me_abs,Delta))
mpe2epe_rel = -b_me_rel*(div_x(u_b_me_rel, Delta)-div_y(v_b_me_rel,Delta))

epe2mpe_rel = u_b_me_rel*div_x(b_me_rel,Delta) + v_b_me_rel*div_y(b_me_rel,Delta)
epe2mpe_abs = u_b_me_abs*div_x(b_me_abs,Delta) + v_b_me_abs*div_y(b_me_abs,Delta)

mpe2epe_abs_i = -integral_z(mpe2epe_abs, dh, Delta, Ht)
mpe2epe_rel_i = -integral_z(mpe2epe_rel, dh, Delta, Ht)

epe2mpe_abs_i = integral_z(epe2mpe_abs, dh, Delta, Ht)
epe2mpe_rel_i = integral_z(epe2mpe_rel, dh, Delta, Ht)

In [38]:
mpe2epe_abs_i_pos = np.where(mpe2epe_abs_i>0,1,0)
mpe2epe_abs_i_neg = 1-mpe2epe_abs_i_pos

mpe2epe_rel_i_pos = np.where(mpe2epe_rel_i>0,1,0)
mpe2epe_rel_i_neg = 1-mpe2epe_rel_i_pos


epe2mpe_abs_i_pos = np.where(epe2mpe_abs_i>0,1,0)
epe2mpe_abs_i_neg = 1-epe2mpe_abs_i_pos

epe2mpe_rel_i_pos = np.where(epe2mpe_rel_i>0,1,0)
epe2mpe_rel_i_neg = 1-epe2mpe_rel_i_pos


mpe2epe_abs_i_pos_i = np.sum(mpe2epe_abs_i_pos*mpe2epe_abs_i)
mpe2epe_abs_i_neg_i = -np.sum(mpe2epe_abs_i_neg*mpe2epe_abs_i)

mpe2epe_rel_i_pos_i = np.sum(mpe2epe_rel_i_pos*mpe2epe_rel_i)
mpe2epe_rel_i_neg_i = -np.sum(mpe2epe_rel_i_neg*mpe2epe_rel_i)


epe2mpe_abs_i_pos_i = np.sum(epe2mpe_abs_i_pos*epe2mpe_abs_i)
epe2mpe_abs_i_neg_i = -np.sum(epe2mpe_abs_i_neg*epe2mpe_abs_i)

epe2mpe_rel_i_pos_i = np.sum(epe2mpe_rel_i_pos*epe2mpe_rel_i)
epe2mpe_rel_i_neg_i = -np.sum(epe2mpe_rel_i_neg*epe2mpe_rel_i)


In [39]:
print('MPE perspective, run abs perte:',mpe2epe_abs_i_neg_i)
print('MPE perspective, run abs gain :',mpe2epe_abs_i_pos_i)
print('net:',mpe2epe_abs_i_neg_i - mpe2epe_abs_i_pos_i)
print('')

print('MPE perspective, run rel perte:',mpe2epe_rel_i_neg_i)
print('MPE perspective, run rel gain :',mpe2epe_rel_i_pos_i)
print('net:',mpe2epe_rel_i_neg_i - mpe2epe_rel_i_pos_i)
print('')

print('EPE perspective, run abs perte:',epe2mpe_abs_i_neg_i)
print('EPE perspective, run abs gain :',epe2mpe_abs_i_pos_i)
print('net:',epe2mpe_abs_i_neg_i - epe2mpe_abs_i_pos_i)
print('')

print('EPE perspective, run rel perte:',epe2mpe_rel_i_neg_i)
print('EPE perspective, run rel gain :',epe2mpe_rel_i_pos_i)
print('net:',epe2mpe_rel_i_neg_i - epe2mpe_rel_i_pos_i)

MPE perspective, run abs perte: 1.2545194404395831e-08
MPE perspective, run abs gain : 9.461105293226943e-09
net: 3.0840891111688887e-09

MPE perspective, run rel perte: 1.040969012505105e-08
MPE perspective, run rel gain : 6.852989643318608e-09
net: 3.556700481732442e-09

EPE perspective, run abs perte: 1.1937664158125748e-08
EPE perspective, run abs gain : 1.3382596421863327e-08
net: -1.4449322637375788e-09

EPE perspective, run rel perte: 9.52962606611084e-09
EPE perspective, run rel gain : 9.385240235680022e-09
net: 1.443858304308184e-10


In [32]:
print(a)
print(b)
print(c)
print(d)

23021384091.29513
23898554414.84832
-49137251469.73025
588700426810.422
